# Wykrywanie encji nazwanych z Flair

To już ostatnie laboratoria zadaniowe, w związku z tym, jeśli znajdziecie chwilę wolnego czasu, wypełnijcie proszę ankietę: https://docs.google.com/forms/d/1rHPjpL70XdXRD-ILl3AHophPNUk0AhsFus1-mtkUPsI

Pozwoli to mi poprawić laboratoria w przyszłości, z góry dziękuję :)

# Flair

Biblioteka Flair to bardzo popularne narzędzie do tagowania sekwencji. Zaintstalujmy ją

In [21]:
!pip install flair

In [22]:
# OPCJONALNE WSPARCIE DLA KART GRAFICZNYCH
# W colabie możemy trenować z wykorzystaniem karty graficznej, dzięki temu trening działa dużo szybciej
# Aby włączyć wsparcie dla karty graficznej musimy:
# 1. w menu 'srodowisko wykonawcze' wybrać `zmien typ srodowiska wykonawczego` i tam `akcelerator sprzętowy` = GPU
# 2. odkomentować linijki poniżej
import flair, torch
flair.device = torch.device('cuda:0') 

# Ładowanie zbioru danych i słownika z etykietami.

**Zadanie1: (1 punkt):** Stwórz słownik etykiet z wczytanego korpusu korzystając z funkcji `make_label_dictionary()`. W naszym zbiorze, etykiety do wykrycia występują w kolumnie `ner`, której identyfikator został zapisany w linijce 6. Tutorial: https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md może okazać się pomocny.

Efektem działania powinna być lista etykiet np: 
`Dictionary with 20 tags: <unk>, Variable, Class, Application, User_Interface_Element, Code_Block, Language, Function, Data_Structure, Library, Data_Type, File_Type, File_Name, Version, HTML_XML_Tag, Device, Operating_System, Website, User_Name, Algorithm`

In [23]:
from flair.datasets import NER_ENGLISH_STACKOVERFLOW    # zbiór otagowanych postów na Stacku

corpus = NER_ENGLISH_STACKOVERFLOW().downsample(0.3)   # pobieramy korpus i zmniejszamy jego wielkość
corpus.filter_empty_sentences()                         # usuwamy puste zdania

label_type = 'ner'   # identyfikator pod którym możemy dostać typy etykiet
label_dict = corpus.make_label_dictionary(label_type=label_type)    # TODO
print('\n\nEtykiety do wykrycia')
print(label_dict)

2022-06-06 12:23:03,478 File train has 741 questions and 897 answers.
2022-06-06 12:23:03,501 File test has 249 questions and 315 answers.
2022-06-06 12:23:03,523 File dev has 247 questions and 289 answers.
2022-06-06 12:23:03,524 Reading data from /home/anna/.flair/datasets/ner_english_stackoverflow
2022-06-06 12:23:03,525 Train: /home/anna/.flair/datasets/ner_english_stackoverflow/train.txt
2022-06-06 12:23:03,526 Dev: /home/anna/.flair/datasets/ner_english_stackoverflow/dev.txt
2022-06-06 12:23:03,527 Test: /home/anna/.flair/datasets/ner_english_stackoverflow/test.txt
2022-06-06 12:23:10,835 Filtering empty sentences
2022-06-06 12:23:10,903 Corpus: 2779 train + 881 dev + 932 test sentences
2022-06-06 12:23:10,904 Computing label dictionary. Progress:


2779it [00:00, 41626.24it/s]

2022-06-06 12:23:10,974 Dictionary created for label 'ner' with 21 values: Application (seen 390 times), Class (seen 374 times), Variable (seen 326 times), User_Interface_Element (seen 290 times), Code_Block (seen 234 times), Function (seen 222 times), Language (seen 216 times), Library (seen 210 times), Data_Structure (seen 189 times), Data_Type (seen 121 times), File_Type (seen 112 times), File_Name (seen 94 times), Version (seen 84 times), Device (seen 71 times), HTML_XML_Tag (seen 57 times), Operating_System (seen 56 times), Website (seen 35 times), User_Name (seen 24 times), Algorithm (seen 10 times), Licence (seen 1 times)


Etykiety do wykrycia
Dictionary with 21 tags: <unk>, Application, Class, Variable, User_Interface_Element, Code_Block, Function, Language, Library, Data_Structure, Data_Type, File_Type, File_Name, Version, Device, HTML_XML_Tag, Operating_System, Website, User_Name, Algorithm, Licence


# Embeddingi

W narzędziu Flair możemy bardzo prosto składać ze sobą różne embeddingi. 

**Zad2 (2 punkty):** Zapoznaj się z działaniem `StackedEmbeddings` opisanego w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md i zbuduj embeddingi zawierające reprezentacje pochodzące zarówno z Glove jak i Flairowe, oparte na `news-forward`. 

In [24]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings

glove_embeddings = WordEmbeddings('glove')
flair_embeddings = FlairEmbeddings('news-forward')


embeddings = StackedEmbeddings([glove_embeddings, flair_embeddings])

# Tagger i trainer

**Zadanie 3 (2 punkty)** Bazując na treściach opisanych w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md przygotuj obiekt `SequenceTagger`, którego rozmiar wartswy ukrytej wyniesie 256. Do obiektu tego przekażemy stworzone wcześniej embeddingi, słownik `label_dict` i nazwę kolumny z etykietą ze zmiennej `label_type`. Ustawmy `use_crf` na True.

Przygotuj obiekt `ModelTrainer`, który przyjmie zarówno nasz korpus jak i stworzony przed chwilą `SequenceTagger`.

In [25]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)
trainer = ModelTrainer(tagger, corpus)


#stworzony trainer możemy zacząć trenować!
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=5)

# a kiedy wytrenujemy, wczytujemy najlepszy model.


2022-06-06 12:23:15,180 SequenceTagger predicts: Dictionary with 81 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Function, B-Function, E-Function, I-Function, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version
2022-06-06 12:23:15,294 ----------------------------------------------------------------------------------------------------
2022-06-06 12:23:15,296 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbed

100%|██████████| 28/28 [00:17<00:00,  1.61it/s]

2022-06-06 12:24:50,611 Evaluating as a multi-label problem: False
2022-06-06 12:24:50,629 DEV : loss 0.5130938291549683 - f1-score (micro avg)  0.0304
2022-06-06 12:24:50,686 BAD EPOCHS (no improvement): 0
2022-06-06 12:24:50,687 saving best model


2022-06-06 12:24:53,665 ----------------------------------------------------------------------------------------------------
2022-06-06 12:24:57,271 epoch 2 - iter 8/87 - loss 0.49748410 - samples/sec: 71.08 - lr: 0.100000
2022-06-06 12:24:59,932 epoch 2 - iter 16/87 - loss 0.52322419 - samples/sec: 96.30 - lr: 0.100000
2022-06-06 12:25:02,400 epoch 2 - iter 24/87 - loss 0.51394017 - samples/sec: 103.85 - lr: 0.100000
2022-06-06 12:25:05,927 epoch 2 - iter 32/87 - loss 0.51684647 - samples/sec: 72.61 - lr: 0.100000
2022-06-06 12:25:08,600 epoch 2 - iter 40/87 - loss 0.50991223 - samples/sec: 95.85 - lr: 0.100000
2022-06-06 12:25:11,718 epoch 2 - iter 48/87 - loss 0.50382130 - samples/sec: 82.15 - lr: 0.100000
2022-06-06 12:25:14,807 epoch 2 - iter 56/87 - loss 0.49602155 - samples/sec: 82.94 - lr: 0.100000
2022-06-06 12:25:17,517 epoch 2 - iter 64/87 - loss 0.50061010 - samples/sec: 94.55 - lr: 0.100000
2022-06-06 12:25:20,905 epoch 2 - iter 72/87 - loss 0.49904033 - samples/sec: 75.60

100%|██████████| 28/28 [00:03<00:00,  7.03it/s]

...

2022-06-06 12:27:25,151 Evaluating as a multi-label problem: False
2022-06-06 12:27:25,173 DEV : loss 0.35304760932922363 - f1-score (micro avg)  0.3028
2022-06-06 12:27:25,234 BAD EPOCHS (no improvement): 0
2022-06-06 12:27:25,236 saving best model


2022-06-06 12:27:29,727 ----------------------------------------------------------------------------------------------------
2022-06-06 12:27:29,729 loading file resources/taggers/example-upos/best-model.pt
2022-06-06 12:27:31,340 SequenceTagger predicts: Dictionary with 83 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Function, B-Function, E-Function, I-Function, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version


100%|██████████| 30/30 [00:18<00:00,  1.64it/s]

2022-06-06 12:27:49,798 Evaluating as a multi-label problem: False
2022-06-06 12:27:49,825 0.3678	0.2247	0.279	0.1894
2022-06-06 12:27:49,826 
Results:
- F-score (micro) 0.279
- F-score (macro) 0.2245
- Accuracy 0.1894

By class:
                        precision    recall  f1-score   support

                 Class     0.3500    0.3913    0.3695       161
           Application     0.2797    0.2391    0.2578       138
              Variable     0.2308    0.0882    0.1277       136
User_Interface_Element     0.4694    0.2018    0.2822       114
               Library     0.2564    0.2632    0.2597        76
            Code_Block     0.1111    0.0814    0.0940        86
              Function     0.4000    0.1500    0.2182        80
              Language     0.5417    0.5000    0.5200        52
        Data_Structure     0.8182    0.4286    0.5625        63
               Version     0.7143    0.3261    0.4478        46
             File_Name     0.3333    0.0185    0.0351        54
 

{'test_score': 0.27896995708154504,
 'dev_score_history': [0.030431107354184278,
  0.04025617566331199,
  0.18352310783657064,
  0.19508670520231214,
  0.3028126870137642],
 'train_loss_history': [0.7335457417961899,
  0.4929056407011975,
  0.42530044657201155,
  0.3826245163520672,
  0.35705954373250837],
 'dev_loss_history': [0.5130938291549683,
  0.4566187560558319,
  0.39059391617774963,
  0.37562450766563416,
  0.35304760932922363]}

In [26]:
model = SequenceTagger.load('resources/taggers/example-upos/final-model.pt')


2022-06-06 12:27:49,918 loading file resources/taggers/example-upos/final-model.pt
2022-06-06 12:27:51,843 SequenceTagger predicts: Dictionary with 83 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Function, B-Function, E-Function, I-Function, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version


# Predykcja z udziałem modelu

Jeśli model został wytrenowany, poniżej znajdziemy fragment kodu, który może wykryć encje w zdaniach.

In [27]:
from flair.data import Sentence

# Jeśli nasz model się wyuczył, powinien wykryć Python jako język.
# Uwaga, ponieważ pracujemy na niewielkim podzbiorze zbioru danych (downsample(0.1) próbkuje 10%), 
# otrzymywane wyniki mogą być kiepskiej jakości, najlepiej zwiększyść ilość danych 
# jeśli pracujemy w domu.
sentence = Sentence('huge files can be opened from Python 3.')   # stwórz obiekt zdania
model.predict(sentence)                                         # wykryj encje nazwane
print(sentence.to_tagged_string())                              # wyświetl zdanie i wykryte w nim encje

Sentence: "huge files can be opened from Python 3 ." → ["Python"/Language, "3"/Version]


In [28]:
sentence = Sentence('Pandas is a library for Python which includes DataFrame class mentioned by Anakin01. Usage: `pd.DataFrame([[1,2],[3,4]])`. You can keep it in variable df_train. It works on both Windows and Linux.')   # stwórz obiekt zdania
model.predict(sentence)                                         # wykryj encje nazwan
print(sentence.to_tagged_string())  

Sentence: "Pandas is a library for Python which includes DataFrame class mentioned by Anakin01 . Usage : ` pd.DataFrame ([[ 1,2 ],[ 3,4 ]])` . You can keep it in variable df _ train . It works on both Windows and Linux ." → ["Python"/Language, "DataFrame"/Class, "Anakin01 . Usage : ` pd.DataFrame ([[ 1,2 ],[ 3,4 ]])`"/Code_Block, "df _ train"/Code_Block, "Windows"/Operating_System, "Linux"/Application]
